In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [11]:
(ds_test, ds_validate, ds_train), ds_info = tfds.load('cats_vs_dogs',
    split=['train[0%:10%]','train[10%:20%]','train[20%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
    data_dir="tensorflow_datasets"
)



In [12]:
print("Train set size: ", len(ds_train)) # Train set size:  18610
print("Test set size: ", len(ds_test))   # Test set size:  2326
print("Valid set size: ", len(ds_validate)) # Valid set size:  2326

Train set size:  18610
Test set size:  2326
Valid set size:  2326


In [13]:
#https://stackoverflow.com/questions/63104794/preprocessing-for-tensorflow-dataset-cats-vs-dogs
NUM_CLASSES=2
IMG_SIZE_W=64
IMG_SIZE_H=64
BATCH_SIZE=64

#Fetching, pre-processing & preparing data-pipeline
def preprocess(ds, label):
    x = tf.image.resize_with_pad(ds['image'], IMG_SIZE_W, IMG_SIZE_H)
    x = tf.cast(x, tf.float32)
    #x = (x-MEAN)/(VARIANCE)
    y = tf.one_hot(ds['label'], NUM_CLASSES)
    return x, y

#def augmentation(image,label):
#    image = tf.image.random_flip_left_right(image)
#    image = tf.image.resize_with_crop_or_pad(image, IMG_W+4, IMG_W+4) # zero pad each side with 4 pixels
#    image = tf.image.random_crop(image, size=[BATCH_SIZE, IMG_W, IMG_H, 3]) # Random crop back to 32x32
#    return image, label

In [14]:
batch_size=64

# Konvertierung von uint8 Bild nach float32 Daten
def normalize_and_resize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

# Aufruf für alle Pixel
ds_train = ds_train.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# Daten in Memory laden
ds_train = ds_train.cache()
# Daten shufflen in Memory 
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)

# Batches herstellen mit jeweils 128 Bildern
ds_train = ds_train.batch(batch_size)
# Vorhalten von Daten für bessere Performanz, Autotune=dynamischer Puffer
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

TypeError: in user code:

    C:\Users\sascha.seifert\AppData\Local\Temp\ipykernel_42024\2568829686.py:9 preprocess  *
        x = tf.image.resize_with_pad(ds['image'], IMG_SIZE_W, IMG_SIZE_H)
    c:\Users\sascha.seifert\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper
        return target(*args, **kwargs)
    c:\Users\sascha.seifert\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\ops\array_ops.py:1013 _slice_helper
        _check_index(s)
    c:\Users\sascha.seifert\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\ops\array_ops.py:887 _check_index
        raise TypeError(_SLICE_TYPE_ERROR + ", got {!r}".format(idx))

    TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got 'image'


In [ ]:
# Normalisierung des validate Datensatzes
ds_validate = ds_validate.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE)
ds_validate = ds_validate.batch(batch_size)
ds_validate = ds_validate.cache()
ds_validate = ds_validate.prefetch(tf.data.AUTOTUNE)

# Normalisierung des test Datensatzes
ds_test = ds_test.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
#l=list(ds_train.as_numpy_iterator())
#plt.imshow(l[0][0][5])
#print("label:",l[0][1][5])


In [ ]:
input_shape = ds_info.features['image'].shape
input_shape
#it = ds_train.as_numpy_iterator()
#el=it.next()

(None, None, 3)

In [ ]:
from ctypes import resize


num_classes = 2
model = keras.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape), # notwendig für summary()
        tf.keras.layers.Conv2D(filters=96, kernel_size=11, strides=4, activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        tf.keras.layers.Conv2D(filters=256, kernel_size=5, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes)
    ]
)
model.summary()



ValueError: The last dimension of the inputs to `Dense` should be defined. Found `None`.

In [ ]:
# model kompilieren und Training starten
from gc import callbacks

max_epochs=2

if model: # auf None prüfen
    model.compile(
        #optimizer=keras.optimizers.Adam(0.005),
        optimizer=keras.optimizers.Adadelta(1.0),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )

    # Callbacks für TensorBoard-Aufzeichnung
    log_dir = "./logs/fit/fashion_mnist_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    model.fit(
        ds_train,
        epochs=max_epochs,
        validation_data=ds_validate,
        callbacks=[tensorboard_callback]
    )


Epoch 1/2
375/375 [==============================] - ETA: 0s - loss: 11.0324 - sparse_categorical_accuracy: 0.0997

KeyboardInterrupt: 